In [5]:
%matplotlib inline
# misc. libraries
import pandas as pd
import numpy as np

# plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

# ml libraries
from sklearn import cluster, metrics
from sklearn.decomposition import PCA

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
# local dependencies
from helpers import *

+ **DHT:** Dihydrotestosterone is an endogenous androgen sex steroid and hormone
+ **E2:** Estradiol (E2), also spelled oestradiol, is an estrogen steroid hormone and the major female sex hormone
+ **P4:**  Progesterone (P4) is an endogenous steroid and progestogen sex hormone involved in the menstrual cycle, pregnancy, and embryogenesis of humans and other species

***
**Data loading**
***

In [ ]:
# Raw information about genes
genes = load_genes()
genes

In [8]:
# Preprocessed list of genes

genes_list = load_genes_list()
genes_list.head(2)

,genes,dht,e2,p4,upregulated
0,ABCA13,False,False,True,True
1,AC062028.2,False,True,False,False


In [ ]:
# Genes showing response to two hormones

genes_list[genes_list[["dht", "e2", "p4"]].sum(axis=1) == 2]

In [ ]:
# Label genes into three categories according to list of genes

genes_labeled = pd.DataFrame(genes_list, columns = ['genes'])
genes_labeled['label'] = genes_list.replace(['dht', 'p4', 'e2'], [0, 1, 2])['label']
genes_labeled.head()

In [9]:
# Load PDX tumor data
pdx = load_pdx(genes_list.genes)

# Remove control subjects and subjects exposed to two treatments
pdx = pdx[~pdx.index.str.contains(r"\+|CTRL")]

# Extract the label from the index
labels = pdx.index.str.rsplit("_", 1).map(lambda x: x[1].lower())
pdx.insert(0, "label", labels)

In [10]:
# Label pdx data according to treatment response
pdx_labeled = pd.DataFrame(genes_list, columns = ['label'])
pdx_labeled = pdx.replace(['dht', 'p4', 'e2'], [0, 1, 2])['label']
pdx_labeled

PL015_G3_M25_E2    2
PL015_G3_M26_E2    2
PL015_G3_M28_P4    1
PL015_G3_M29_P4    1
PL015_G3_M33_E2    2
PL015_G3_M35_P4    1
PL015_G3_M38_P4    1
PL015_G3_M39_E2    2
T110_G2_M03_E2     2
T110_G2_M05_P4     1
T110_G2_M07_P4     1
T110_G2_M09_E2     2
T110_G2_M12_P4     1
T110_G2_M13_P4     1
T111_G2_M02_DHT    0
T111_G2_M03_E2     2
T111_G2_M06_DHT    0
T111_G2_M07_P4     1
T111_G2_M09_DHT    0
T111_G2_M10_E2     2
T111_G2_M12_E2     2
T111_G2_M14_P4     1
T111_G2_M15_P4     1
Name: label, dtype: int64

In [ ]:
# Load TCGA patient data (first part)
patients = load_patients(genes_list.genes)
patients.head(2)

In [ ]:
# Load TCGA patient data (second part)
patients2 = load_patients2(genes_list.genes)
patients2.head(2)

***
**Exploratory data analysis**
***

In [ ]:
corr = np.abs(np.tril(pdx.corr()))

plt.figure(figsize=(25, 25))
ticklabels = genes_list.genes

heatmap = sns.heatmap(
    corr,
    square=True,
    linewidths=.005,
    xticklabels=ticklabels,
    yticklabels=ticklabels,
    mask=(corr == 0)  # mask cells with missing values
)
fig = heatmap.get_figure()
fig.savefig("corr.png")

***
**Feature processing**
***

In [ ]:
# todo

***
**Clustering**
***

In [ ]:
clus = cluster.AgglomerativeClustering(n_clusters=7)#, affinity='manhattan', linkage='average')
predicted = clus.fit_predict(pdx.drop(labels='label', axis=1))
print(pdx_labeled.shape)

# calculate score
score = metrics.adjusted_rand_score(pdx_labeled, predicted)
print(score)
#accuracy, f2 = performance(predicted)

***
**Spectral Clustering**
***

In [23]:
clustering = cluster.SpectralClustering(assign_labels="discretize", random_state=0).fit(pdx.drop(labels='label', axis=1))
print("predicted labels : " + str(clustering.labels_))
print("true labels :      " + str(pdx_labeled.values))
print("Score : " + str(metrics.adjusted_rand_score(pdx_labeled, clustering.labels_)))

predicted labels : [4 4 2 2 4 2 2 4 4 0 0 4 0 2 3 0 7 1 7 0 0 0 0]
true labels :      [2 2 1 1 2 1 1 2 2 1 1 2 1 1 0 2 0 1 0 2 2 1 1]
Score : 0.35109189506082367


***
**K-Means**
***

In [ ]:
kmeans = cluster.KMeans(n_clusters=4, random_state=0).fit(pdx.drop(labels='label', axis=1))
print("predicted labels : " + str(kmeans.labels_))
print("true labels :      " + str(pdx_labeled.values))
print("Score : " + str(metrics.adjusted_rand_score(pdx_labeled, kmeans.labels_)))

In [ ]:
# we should rather evaluate with the metrics.adjusted_rand_score function 

def performance(labels):
    '''Evaluate performance of predicted cluster compared to pre-selected gene list'''
    # get gene list
    geneNP = genes_list.loc[:,'dht':'p4'].astype(int).values  # replace with Boolean values
    
    nb_clusters = len(np.unique(labels))
    accuracy = np.zeros([nb_clusters, 3])
    f2 =  np.zeros([nb_clusters, 3])
    beta = 2
    for i in np.arange(nb_clusters):
        label = np.zeros_like(labels)
        label[labels == i] = 1
        for j in np.arange(geneNP.shape[1]):
            # plot confusion matrices
            
            #cm = metrics.confusion_matrix(geneNP[:,j], label)
            #cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
            #fig, ax = plt.subplots()
            #im = ax.imshow(cm, interpolation='nearest')
            #ax.figure.colorbar(im, ax=ax)
            accuracy[i,j] = np.mean(geneNP[:,j]==label)
            f2[i,j] = metrics.fbeta_score(geneNP[:,j], label, beta)
    return accuracy, f2
        

***
**Principal Component Analysis**
***

In [ ]:
# PCA decomposition of original gene list
# we want to verify that the pre-selected genes are linearly independent

pca = PCA()
pca.fit(genes_list)
PCA(copy=True, iterated_power='auto', n_components=None,
    random_state=None, svd_solver='auto', tol=0.0, whiten=False)

# Explained variance by PCA
# print(pca.explained_variance_)

# Singular values of PCA
# print(pca.singular_values_)

y_pos = np.arange(len(pca.singular_values_))
plt.bar(y_pos, pca.singular_values_, align='center', alpha=0.5)
plt.ylabel('Values')
plt.xlabel('Principal components')
plt.title('PCA - Singular values')
plt.show()

#
#pca.n_components = 2
#X_reduced = pca.fit_transform(X)
# X_reduced.shape